In [18]:
import pandas as pd
import numpy as np
import sqlite3
import itertools

pd.set_option('display.max_columns', 1000)

In [19]:
conn = sqlite3.connect('./database.sqlite')

# 1. EDA

In [20]:
pd.read_sql('SELECT * FROM sqlite_master where type="table"', conn)

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [21]:
matches_df = pd.read_sql('SELECT * FROM Match', conn)
players_df = pd.read_sql('SELECT * FROM Player', conn)
leagues_df = pd.read_sql('SELECT * FROM League', conn)
country_df = pd.read_sql('SELECT * FROM Country', conn)
team_df = pd.read_sql('SELECT * FROM Team', conn)
team_attributes_df = pd.read_sql('SELECT * FROM Team_Attributes', conn)
player_attributes_df = pd.read_sql('SELECT * FROM Player_Attributes', conn)

## How many total matches are there?

In [22]:
matches_df.shape

(25979, 115)

### What are the earliest matches we have?

In [23]:
matches_df.sort_values(by='date', ascending=True).iloc[0:5, :]

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
24558,24559,24558,24558,2008/2009,1,2008-07-18 00:00:00,486263,10192,9931,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24559,24560,24558,24558,2008/2009,1,2008-07-19 00:00:00,486264,9930,10179,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24560,24561,24558,24558,2008/2009,1,2008-07-20 00:00:00,486265,10199,9824,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24561,24562,24558,24558,2008/2009,1,2008-07-20 00:00:00,486266,7955,10243,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24613,24614,24558,24558,2008/2009,2,2008-07-23 00:00:00,486268,6493,7955,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Is there enough player data per match? Print a dataframe showing the number of missing rows per column

In [24]:
# matches_df = matches_df.dropna(subset=[f'home_player_{i}' for i in range(1, 12)], axis=0)
# matches_df = matches_df.dropna(subset=[f'away_player_{i}' for i in range(1, 12)], axis=0)
for col in matches_df.columns:
    if f'home_player_' in col:
        matches_df[col] = matches_df[col].fillna(matches_df[col].mean())
matches_df = matches_df.iloc[:, :-30] # remove betting features

dic = dict(zip(matches_df.columns, matches_df.isna().sum()))
na_df = pd.DataFrame([dic])
na_df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1832,1832,1832,1832,1832,1832,1832,1832,1833,1833,1839,0,0,0,0,0,0,0,0,0,0,0,1832,1832,1832,1832,1832,1832,1832,1832,1833,1833,1839,0,0,0,0,0,0,0,0,0,0,0,1234,1278,1293,1321,1335,1313,1235,1341,1328,1441,1554,11762,11762,11762,11762,11762,11762,11762,11762


In [25]:
# merge matches dataframe with team attributes frame by team ID and season
def get_season(date):
    year, month = date.split('-')[0], date.split('-')[1]
    # typical season runs from august to may
    if (int(month) < 8):
        return f'{int(year)-1}/{year}'
    return f'{year}/{int(year)+1}'

team_attributes_df['season_new'] = team_attributes_df['date'].apply(get_season)
# There are duplicate seasons for some players
matches_df = matches_df.merge(team_attributes_df, left_on=['home_team_api_id','season'], right_on = ['team_api_id','season_new'], how='left')
matches_df = matches_df.merge(team_attributes_df, left_on=['away_team_api_id','season'], right_on = ['team_api_id','season_new'], how='left')
# matches_df = matches_df.drop(['home_team_api_id', 'team_api_id', 'match_id', 'date', 'match_api_id', 'away_team_api_id'])
matches_df.head()

,id_x,country_id,league_id,season,stage,date_x,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,id_y,team_fifa_api_id_x,team_api_id_x,date_y,buildUpPlaySpeed_x,buildUpPlaySpeedClass_x,buildUpPlayDribbling_x,buildUpPlayDribblingClass_x,buildUpPlayPassing_x,buildUpPlayPassingClass_x,buildUpPlayPositioningClass_x,chanceCreationPassing_x,chanceCreationPassingClass_x,chanceCreationCrossing_x,chanceCreationCrossingClass_x,chanceCreationShooting_x,chanceCreationShootingClass_x,chanceCreationPositioningClass_x,defencePressure_x,defencePressureClass_x,defenceAggression_x,defenceAggressionClass_x,defenceTeamWidth_x,defenceTeamWidthClass_x,defenceDefenderLineClass_x,season_new_x,id,team_fifa_api_id_y,team_api_id_y,date,buildUpPlaySpeed_y,buildUpPlaySpeedClass_y,buildUpPlayDribbling_y,buildUpPlayDribblingClass_y,buildUpPlayPassing_y,buildUpPlayPassingClass_y,buildUpPlayPositioningClass_y,chanceCreationPassing_y,chanceCreationPassingClass_y,chanceCreationCrossing_y,chanceCreationCrossingClass_y,chanceCreationShooting_y,chanceCreationShootingClass_y,chanceCreationPositioningClass_y,defencePressure_y,defencePressureClass_y,defenceAggression_y,defenceAggressionClass_y,defenceTeamWidth_y,defenceTeamWidthClass_y,defenceDefenderLineClass_y,season_new_y
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,0.999586,2.073516,4.061001,6.049199,7.544871,3.185158,4.769909,5.309604,5.822048,5.389407,5.783244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999627,2.998634,3.000083,3.000414,3.237214,6.476954,6.672257,7.238953,8.026339,9.218868,10.43699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76638.362432,106854.091996,91601.289457,94540.204494,109527.842071,102308.768516,97287.636918,107290.992947,111131.536914,105612.508373,103413.87066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,0.999586,2.073516,4.061001,6.049199,7.544871,3.185158,4.769909,5.309604,5.822048,5.389407,5.783244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999627,2.998634,3.000083,3.000414,3.237214,6.476954,6.672257,7.238953,8.026339,9.218868,10.43699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76638.362432,106854.091996,91601.289457,94540.204494,109527.842071,102308.768516,97287.636918,107290.992947,111131.536914,105612.508373,103413.87066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,0.999586,2.073516,4.061001,6.049199,7.54

In [26]:
player_attributes_df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [27]:
# merge matches_df with player_attributes_df
player_attributes_df['season'] = player_attributes_df['date'].apply(get_season)
player_attributes_df = player_attributes_df.drop_duplicates(subset=['player_api_id', 'season'])
for i in range(1, 12):
    matches_df = matches_df.merge(player_attributes_df, left_on=[f'home_player_{i}','season'], right_on = ['player_api_id','season'], how='left', suffixes=[f'_home_{i}_l', f'_home_{i}_r'], validate='many_to_one')
    matches_df = matches_df.merge(player_attributes_df, left_on=[f'away_player_{i}','season'], right_on = ['player_api_id','season'], how='left', suffixes=[f'_away_{i}_l', f'_away_{i}_r'], validate='many_to_one')


/home/rufael.marew/.conda/envs/ml701/lib/python3.6/site-packages/pandas/core/reshape/merge.py:1117: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


In [28]:
dict(matches_df.isna().sum())

{'id_x': 0,
 'country_id': 0,
 'league_id': 0,
 'season': 0,
 'stage': 0,
 'date_x': 0,
 'match_api_id': 0,
 'home_team_api_id': 0,
 'away_team_api_id': 0,
 'home_team_goal': 0,
 'away_team_goal': 0,
 'home_player_X1': 0,
 'home_player_X2': 0,
 'home_player_X3': 0,
 'home_player_X4': 0,
 'home_player_X5': 0,
 'home_player_X6': 0,
 'home_player_X7': 0,
 'home_player_X8': 0,
 'home_player_X9': 0,
 'home_player_X10': 0,
 'home_player_X11': 0,
 'away_player_X1': 1832,
 'away_player_X2': 1832,
 'away_player_X3': 1832,
 'away_player_X4': 1832,
 'away_player_X5': 1832,
 'away_player_X6': 1832,
 'away_player_X7': 1832,
 'away_player_X8': 1832,
 'away_player_X9': 1833,
 'away_player_X10': 1833,
 'away_player_X11': 1839,
 'home_player_Y1': 0,
 'home_player_Y2': 0,
 'home_player_Y3': 0,
 'home_player_Y4': 0,
 'home_player_Y5': 0,
 'home_player_Y6': 0,
 'home_player_Y7': 0,
 'home_player_Y8': 0,
 'home_player_Y9': 0,
 'home_player_Y10': 0,
 'home_player_Y11': 0,
 'away_player_Y1': 1832,
 'away_pla

In [29]:
dic = dict(matches_df.isna().sum())
for key in dic:
    if (dic[key] / len(matches_df) > 0.3).any():
        matches_df = matches_df.drop(key, axis=1)

In [30]:
num_cols = matches_df.select_dtypes(include=['int','float', 'float64']).columns
cat_cols = [x for x in matches_df.columns if x not in num_cols]

for col in num_cols:
    if col[0:-2] in team_attributes_df.columns and col.find('_x') != -1:
        matches_df[col] = matches_df[col].fillna(matches_df.groupby('home_team_api_id')[col].transform('mean'))
    elif col[0:-2] in team_attributes_df.columns and '_y' in col:
        matches_df[col] = matches_df[col].fillna(matches_df.groupby('away_team_api_id')[col].transform('mean'))
   
# f = lambda x: x.median() if np.issubdtype(x.dtype, np.number) else x.mode()
f = lambda x: x.mode().iloc[0] if len(x.mode())>0 else np.NaN
matches_df[cat_cols] = matches_df[cat_cols].fillna(matches_df.groupby('home_team_api_id').transform(f))
matches_df[cat_cols] = matches_df[cat_cols].fillna(matches_df.groupby('away_team_api_id').transform(f))

for col in cat_cols:
    matches_df[col] = matches_df[col].astype('category').cat.codes
matches_df.head()

,id_x,country_id,league_id,season,stage,date_x,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,id_y,team_fifa_api_id_x,team_api_id_x,date_y,buildUpPlaySpeed_x,buildUpPlaySpeedClass_x,buildUpPlayDribblingClass_x,buildUpPlayPassing_x,buildUpPlayPassingClass_x,buildUpPlayPositioningClass_x,chanceCreationPassing_x,chanceCreationPassingClass_x,chanceCreationCrossing_x,chanceCreationCrossingClass_x,chanceCreationShooting_x,chanceCreationShootingClass_x,chanceCreationPositioningClass_x,defencePressure_x,defencePressureClass_x,defenceAggression_x,defenceAggressionClass_x,defenceTeamWidth_x,defenceTeamWidthClass_x,defenceDefenderLineClass_x,season_new_x,id_home_1_l,team_fifa_api_id_y,team_api_id_y,date_home_1_l,buildUpPlaySpeed_y,buildUpPlaySpeedClass_y,buildUpPlayDribblingClass_y,buildUpPlayPassing_y,buildUpPlayPassingClass_y,buildUpPlayPositioningClass_y,chanceCreationPassing_y,chanceCreationPassingClass_y,chanceCreationCrossing_y,chanceCreationCrossingClass_y,chanceCreationShooting_y,chanceCreationShootingClass_y,chanceCreationPositioningClass_y,defencePressure_y,defencePressureClass_y,defenceAggression_y,defenceAggressionClass_y,defenceTeamWidth_y,defenceTeamWidthClass_y,defenceDefenderLineClass_y,season_new_y,id_home_1_r,player_fifa_api_id_away_1_l,player_api_id_away_1_l,date_home_1_r,overall_rating_away_1_l,potential_away_1_l,preferred_foot_away_1_l,attacking_work_rate_away_1_l,defensive_work_rate_away_1_l,crossing_away_1_l,finishing_away_1_l,heading_accuracy_away_1_l,short_passing_away_1_l,volleys_away_1_l,dribbling_away_1_l,curve_away_1_l,free_kick_accuracy_away_1_l,long_passing_away_1_l,ball_control_away_1_l,acceleration_away_1_l,sprint_speed_away_1_l,agility_away_1_l,reactions_away_1_l,balance_away_1_l,shot_power_away_1_l,jumping_away_1_l,stamina_away_1_l,strength_away_1_l,long_shots_away_1_l,aggression_away_1_l,interceptions_away_1_l,positioning_away_1_l,vision_away_1_l,penalties_away_1_l,marking_away_1_l,standing_tackle_away_1_l,sliding_tackle_away_1_l,gk_diving_away_1_l,gk_handling_away_1_l,gk_kicking_away_1_l,gk_positioning_away_1_l,gk_reflexes_away_1_l,id_home_2_l,player_fifa_api_id_away_1_r,player_api_id_away_1_r,date_home_2_l,overall_rating_away_1_r,potential_away_1_r,preferred_foot_away_1_r,attacking_work_rate_away_1_r,defensive_work_rate_away_1_r,crossing_away_1_r,finishing_away_1_r,heading_accuracy_away_1_r,short_passing_away_1_r,volleys_away_1_r,dribbling_away_1_r,curve_away_1_r,free_kick_accuracy_away_1_r,long_passing_away_1_r,ball_control_away_1_r,acceleration_away_1_r,sprint_speed_away_1_r,agility_away_1_r,reactions_away_1_r,balance_away_1_r,shot_power_away_1_r,jumping_away_1_r,stamina_away_1_r,strength_away_1_r,long_shots_away_1_r,aggression_away_1_r,interceptions_away_1_r,positioning_away_1_r,vision_away_1_r,penalties_away_1_r,marking_away_1_r,standing_tackle_away_1_r,sliding_tackle_away_1_r,gk_diving_away_1_r,gk_handling_away_1_r,gk_kicking_away_1_r,gk_positioning_away_1_r,gk_reflexes_away_1_r,id_home_2_r,player_fifa_ap

In [34]:
num_cols = matches_df.select_dtypes(include=['int','float', 'float64']).columns
cat_cols = [x for x in matches_df.columns if x not in num_cols]

for col in num_cols:
    if col[0:-2] in player_attributes_df.columns and col.find('_l') != -1:
        for i in range(1, 12):
            matches_df[col] = matches_df[col].fillna(matches_df.groupby(f'home_player_{i}')[col].transform('mean'))
    elif col[0:-2] in player_attributes_df.columns and '_r' in col:
        for i in range(1, 12):
            matches_df[col] = matches_df[col].fillna(matches_df.groupby(f'away_player_{i}')[col].transform('mean'))
   
# f = lambda x: x.median() if np.issubdtype(x.dtype, np.number) else x.mode()
f = lambda x: x.mode().iloc[0] if len(x.mode())>0 else np.NaN
for col in num_cols:
    if col[0:-2] in player_attributes_df.columns and col.find('_l') != -1:
        for i in range(1, 12):
            matches_df[col] = matches_df[col].fillna(matches_df.groupby(f'home_player_{i}')[col].transform(f))
    elif col[0:-2] in player_attributes_df.columns and '_r' in col:
        for i in range(1, 12):
            matches_df[col] = matches_df[col].fillna(matches_df.groupby(f'away_player_{i}')[col].transform(f))
for col in cat_cols:
    matches_df[col] = matches_df[col].astype('category').cat.codes
matches_df.head()

,id_x,country_id,league_id,season,stage,date_x,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,id_y,team_fifa_api_id_x,team_api_id_x,date_y,buildUpPlaySpeed_x,buildUpPlaySpeedClass_x,buildUpPlayDribblingClass_x,buildUpPlayPassing_x,buildUpPlayPassingClass_x,buildUpPlayPositioningClass_x,chanceCreationPassing_x,chanceCreationPassingClass_x,chanceCreationCrossing_x,chanceCreationCrossingClass_x,chanceCreationShooting_x,chanceCreationShootingClass_x,chanceCreationPositioningClass_x,defencePressure_x,defencePressureClass_x,defenceAggression_x,defenceAggressionClass_x,defenceTeamWidth_x,defenceTeamWidthClass_x,defenceDefenderLineClass_x,season_new_x,id_home_1_l,team_fifa_api_id_y,team_api_id_y,date_home_1_l,buildUpPlaySpeed_y,buildUpPlaySpeedClass_y,buildUpPlayDribblingClass_y,buildUpPlayPassing_y,buildUpPlayPassingClass_y,buildUpPlayPositioningClass_y,chanceCreationPassing_y,chanceCreationPassingClass_y,chanceCreationCrossing_y,chanceCreationCrossingClass_y,chanceCreationShooting_y,chanceCreationShootingClass_y,chanceCreationPositioningClass_y,defencePressure_y,defencePressureClass_y,defenceAggression_y,defenceAggressionClass_y,defenceTeamWidth_y,defenceTeamWidthClass_y,defenceDefenderLineClass_y,season_new_y,id_home_1_r,player_fifa_api_id_away_1_l,player_api_id_away_1_l,date_home_1_r,overall_rating_away_1_l,potential_away_1_l,preferred_foot_away_1_l,attacking_work_rate_away_1_l,defensive_work_rate_away_1_l,crossing_away_1_l,finishing_away_1_l,heading_accuracy_away_1_l,short_passing_away_1_l,volleys_away_1_l,dribbling_away_1_l,curve_away_1_l,free_kick_accuracy_away_1_l,long_passing_away_1_l,ball_control_away_1_l,acceleration_away_1_l,sprint_speed_away_1_l,agility_away_1_l,reactions_away_1_l,balance_away_1_l,shot_power_away_1_l,jumping_away_1_l,stamina_away_1_l,strength_away_1_l,long_shots_away_1_l,aggression_away_1_l,interceptions_away_1_l,positioning_away_1_l,vision_away_1_l,penalties_away_1_l,marking_away_1_l,standing_tackle_away_1_l,sliding_tackle_away_1_l,gk_diving_away_1_l,gk_handling_away_1_l,gk_kicking_away_1_l,gk_positioning_away_1_l,gk_reflexes_away_1_l,id_home_2_l,player_fifa_api_id_away_1_r,player_api_id_away_1_r,date_home_2_l,overall_rating_away_1_r,potential_away_1_r,preferred_foot_away_1_r,attacking_work_rate_away_1_r,defensive_work_rate_away_1_r,crossing_away_1_r,finishing_away_1_r,heading_accuracy_away_1_r,short_passing_away_1_r,volleys_away_1_r,dribbling_away_1_r,curve_away_1_r,free_kick_accuracy_away_1_r,long_passing_away_1_r,ball_control_away_1_r,acceleration_away_1_r,sprint_speed_away_1_r,agility_away_1_r,reactions_away_1_r,balance_away_1_r,shot_power_away_1_r,jumping_away_1_r,stamina_away_1_r,strength_away_1_r,long_shots_away_1_r,aggression_away_1_r,interceptions_away_1_r,positioning_away_1_r,vision_away_1_r,penalties_away_1_r,marking_away_1_r,standing_tackle_away_1_r,sliding_tackle_away_1_r,gk_diving_away_1_r,gk_handling_away_1_r,gk_kicking_away_1_r,gk_positioning_away_1_r,gk_reflexes_away_1_r,id_home_2_r,player_fifa_ap

In [35]:
matches_df['label'] = (matches_df['home_team_goal'] - matches_df['away_team_goal']).apply(np.sign)
drop_cols = ['home_team_goal','away_team_goal','goal','date','shoton',
             'shotoff','foulcommit','card','cross','corner','possession',
             'match_api_id', 'season_new_x', 'id', 'season_new_y', 'id_y',
             'team_api_id_x', 'team_fifa_api_id_x', 'team_fifa_api_id_y',
             'team_api_id_y', 'id_x', 'buildUpPlayDribbling_x', 'home_team_api_id']
matches_df.drop(drop_cols, inplace=True, axis=1, errors='ignore')
matches_df.drop(['date_x','date_y'], inplace=True, axis=1, errors='ignore')
matches_df['season'] = matches_df['season'].astype('category').cat.codes
for col in matches_df.columns:
    matches_df[col] = matches_df[col].fillna(matches_df[col].mean())

In [36]:
# How many matches per league do we have?
# matches_df.groupby('country_id').size()
# matches_df[['season', 'home_team_api_id']].nunique(), team_attributes_df[['season_new', 'team_api_id']].nunique()
dict(matches_df.isna().sum())

{'country_id': 0,
 'league_id': 0,
 'season': 0,
 'stage': 0,
 'away_team_api_id': 0,
 'home_player_X1': 0,
 'home_player_X2': 0,
 'home_player_X3': 0,
 'home_player_X4': 0,
 'home_player_X5': 0,
 'home_player_X6': 0,
 'home_player_X7': 0,
 'home_player_X8': 0,
 'home_player_X9': 0,
 'home_player_X10': 0,
 'home_player_X11': 0,
 'away_player_X1': 0,
 'away_player_X2': 0,
 'away_player_X3': 0,
 'away_player_X4': 0,
 'away_player_X5': 0,
 'away_player_X6': 0,
 'away_player_X7': 0,
 'away_player_X8': 0,
 'away_player_X9': 0,
 'away_player_X10': 0,
 'away_player_X11': 0,
 'home_player_Y1': 0,
 'home_player_Y2': 0,
 'home_player_Y3': 0,
 'home_player_Y4': 0,
 'home_player_Y5': 0,
 'home_player_Y6': 0,
 'home_player_Y7': 0,
 'home_player_Y8': 0,
 'home_player_Y9': 0,
 'home_player_Y10': 0,
 'home_player_Y11': 0,
 'away_player_Y1': 0,
 'away_player_Y2': 0,
 'away_player_Y3': 0,
 'away_player_Y4': 0,
 'away_player_Y5': 0,
 'away_player_Y6': 0,
 'away_player_Y7': 0,
 'away_player_Y8': 0,
 'away

In [37]:
from sklearn.model_selection import train_test_split

X = matches_df.iloc[:,:-1]
y = matches_df.iloc[:,-1]
y.replace({-1:2,1:1,0:0},inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=0)

In [38]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier(random_state=0)

print(model.fit(X_train,y_train))
y_temp = model.predict(X_train)
print(accuracy_score(y_train, y_temp))
y_pred = model.predict(X_test)

accuracy_score(y_pred,y_test)

DecisionTreeClassifier(random_state=0)
1.0


0.3968437259430331

In [ ]:
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier 
from sklearn.model_selection import GridSearchCV

params = {
        'min_child_weight': [1, 5, 10],
        'subsample': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5,10,15]
        }

model_xgb = XGBClassifier()
model_grid = GridSearchCV(model_xgb, params, n_jobs=-1, verbose=2)


model_grid.fit(X_train,y_train)
model_grid.cv_results_
# y_pred = model_xgb.predict(X_test)
# accuracy_score(y_pred,y_test)

In [43]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier()
model_xgb.fit(X_train,y_train,verbose=4)
y_pred = model_xgb.predict(X_test)
accuracy_score(y_pred,y_test)

/home/rufael.marew/.conda/envs/ml701/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:33:18] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.4897998460354119

In [41]:
!pip install xgboost

     |################################| 173.6 MB 21 kB/s  eta 0:00:012   |##                              | 12.9 MB 9.0 MB/s eta 0:00:18


In [ ]:
matches_df.head()

In [ ]:
y_tem = model_xgb.predict(X_train)
accuracy_score(y_train, y_tem)

In [ ]:
from sklearn.svm import SVC


model_svc = SVC()
param_svc = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
             } 

model_grid_svm = GridSearchCV(model_svc,param_svc, n_jobs=-1)
model_grid_svm.fit(X_train, y_train)
model_grid.cv_results_
# accuracy_score(model_svc.predict(X_test), y_test)